In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from datetime import date, timedelta
import matplotlib.dates as mdates

c:\Users\prath\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
url = "historic_data/csv/TCS.NS.csv"    
col_names = ['Date','Open','High','Low','Close','Volume','Adj Close']
stocks = pd.read_csv(url, header=0, names=col_names) 
df = pd.DataFrame(stocks).dropna(how='any', axis=0)
date_split = df['Date'].str.split('-').str
df['Year'], df['Month'], df['Day'] = date_split
df["Volume"] = df["Volume"] / 10000

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')


#sort by date
df = df.sort_values('Date')

# fix the date 
df.reset_index(inplace=True)
df.set_index("Date", inplace=True)

dates_df = df.copy()
dates_df = dates_df.reset_index()

dates_df['Date'] = pd.to_datetime(dates_df['Date'])


# Store the original dates for plotting the predicitons
org_dates = dates_df['Date']

# convert to ints
dates_df['Date'] = dates_df['Date'].map(mdates.date2num)
dates_df.head()
dates_df.tail()

dates_df.tail()

C:\Users\prath\AppData\Local\Temp\ipykernel_18932\461335973.py:6: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Year'], df['Month'], df['Day'] = date_split


,Date,index,Open,High,Low,Close,Volume,Adj Close,Year,Month,Day
5141,19472.0,5150,3147.500000,3183.000000,2024090.0,3191.199951,0.317600,3176.000000,25,04,2023
5142,19473.0,5151,3181.000000,3181.000000,2000029.0,3208.399902,0.319815,3198.149902,26,04,2023
5143,19474.0,5152,3170.649902,3185.000000,2434446.0,3199.199951,0.318795,3187.949951,27,04,2023
5144,19475.0,5153,3196.250000,3200.399902,2931312.0,3227.250000,0.321925,3219.250000,28,04,2023
5145,19479.0,5154,3210.000000,3235.000000,2320980.0,3235.000000,0.321940,3219.399902,02,05,2023


In [3]:
X = dates_df.Date
y = dates_df.Close

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(y_test)

4116    2074.949951
4117    2071.350098
4118    2005.000000
4119    2007.000000
4120    2015.000000
           ...     
5141    3191.199951
5142    3208.399902
5143    3199.199951
5144    3227.250000
5145    3235.000000
Name: Close, Length: 1030, dtype: float64


In [4]:
paramters = {
        'C':[0.001,0.01,0.1,1,100,1000],
        'epsilon': [
                    0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,
                    50, 100, 150, 1000
                ],
        'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5, 8, 40, 100, 1000]
    }

In [5]:
import numpy as np
from datetime import timedelta, date
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR

def predict_stock_prices_graph(X_train, y_train, n_days):
    paramters = {
        'C': [0.1,1, 10, 100],
        'gamma': [1,0.1,0.01,0.001],
        'epsilon': [0.1,0.01,0.001,0.0001]
    }

    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=paramters,
        cv=10,
        scoring='neg_mean_absolute_error',
        error_score='raise'
    )

    X_train = np.arange(len(X_train)).reshape(-1, 1)
    y_train = np.array(y_train).reshape(-1, 1)

    grid_result = gsc.fit(X_train, y_train.ravel())

    # storing and using best parameters
    # best parameters = min error
    best_param = grid_result.best_params_
    svr_model = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma'])
    svr_model.fit(X_train, y_train.ravel())

    output_days = np.arange(len(X_train), len(X_train)+n_days).reshape(-1, 1)

    dates = []
    current = date.today()
    for i in range(n_days): # creating timeline for future dates
        current += timedelta(days=1)
        dates.append(current)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
                x=dates, 
                y=svr_model.predict(output_days),
                mode='lines+markers',
                name='data'))
    fig.update_layout(
        title="Predicted Close Price of next " + str(n_days) + " days",
        xaxis_title="Date",
        yaxis_title="Closed Price"
    )

    return fig


In [6]:
predict_stock_prices_graph(X_test, y_test,60)

In [17]:
def predict_stock_prices_graph(X_train, y_train, n_days):
    paramters = {
        'C': [0.1,1, 10, 100],
        'gamma': [1,0.1,0.01,0.001],
        'epsilon': [0.1,0.01,0.001,0.0001]
    }

    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=paramters,
        cv=10,
        scoring='neg_mean_absolute_error',
        error_score='raise'
    )

    X_train = np.arange(len(X_train)).reshape(-1, 1)
    y_train = np.array(y_train).reshape(-1, 1)

    grid_result = gsc.fit(X_train, y_train.ravel())

    # storing and using best parameters
    # best parameters = min error
    best_param = grid_result.best_params_
    svr_model = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma'])
    svr_model.fit(X_train, y_train.ravel())

    output_days = np.arange(len(X_train), len(X_train)+n_days).reshape(-1, 1)

    dates = []
    current = date.today()
    for i in range(n_days): # creating timeline for future dates
        current += timedelta(days=1)
        dates.append(current)

    predicted_prices = svr_model.predict(output_days)
    return predicted_prices



In [18]:
svr_prices = predict_stock_prices_graph(X_test, y_test, 2)


In [19]:
print(svr_prices)

[3185.14377411 3184.64686791]


In [21]:
import pickle

# Assuming that you have already executed the code to get the values
# Store the values in a dictionary
values_dict = {
    'svr1': svr_prices[0],
    'svr2': svr_prices[1],
}

# Save the dictionary to a file using pickle
with open('svr.pkl', 'wb') as f:
    pickle.dump(values_dict, f)
